# Imports

In [1]:
from pulp import *
import pandas as pd

import pulp

# References

https://www2.census.gov/geo/docs/reference/county_adjacency.txt

https://worldpopulationreview.com/us-counties

https://www.elections.il.gov/electionoperations/votetotalsearch.aspx

https://coin-or.github.io/pulp/CaseStudies/a_set_partitioning_problem.html



# Set Up

In [2]:
# population data

pop_data = pd.read_csv('/Users/gracefujinaga/Documents/Northwestern/MSDS_460/redistricting/pop_data.csv')
county_population = pop_data.set_index('county')['pop2024'].to_dict()
county_population

total_pop = pop_data['pop2024'].sum()

In [3]:
# adjacency list

def parse_illinois_counties_to_adjacency_list(file_path):
    adjacency_list = {}
    current_county = None
    
    with open(file_path, 'r') as file:
        for line in file:
            # Check if it's a main county line (no leading whitespace) and contains "IL"
            if line.startswith('"') and "IL" in line:
                # Extract the main county name
                county_name = line.split("\t")[0].strip('"')
                # Initialize an empty list of neighbors for the current county
                adjacency_list[county_name] = []
                current_county = county_name
            elif current_county and "IL" in line:
                # Only add neighboring counties in Illinois
                neighbor_county = line.strip().split("\t")[0].strip('"')
                adjacency_list[current_county].append(neighbor_county)
    
    return adjacency_list

# Example usage

adj_list = parse_illinois_counties_to_adjacency_list('/Users/gracefujinaga/Documents/Northwestern/MSDS_460/redistricting/Data/adjacency_data.txt')

In [4]:
# define counties
counties = list(pop_data.county)

# define districts
districts = [str(i) for i in range(1,18)]

# LP Problem

In [21]:
#create dictionary for county names with their population sizes, adjacency, latitude, longitude, etc
county_dict = {
    row['name']: {
        "pop2024": row['pop2024'],
        "pop2020": row['pop2020'],
        "adjacentcodes": row['AdjacentCodes']
    }
    for _, row in county_data_df.iterrows()
}

#max_districts
max_districts = 17
#max_districts size? (max counties in district)

#list of all possible districts
possible_districts = [tuple(c) for c in pulp.allcombinations(counties, max_population_size)]
#would be better to use max/min population sizes?

#create binary variable to state county is in a district
x = pulp.LpVariable.dicts(
    "table", possible_districts, lowBound=0, upBound=1, cat=pulp.LpInteger
)
redistprob = pulp.LpProblem("Redistrict_probModel", pulp.LpMinimize)

#distance function (used in obj function)
#GIS distance calculations code from the assignment page
#requires latitude and longitude

#obj function, Sum of population per district is as close to equal as possible


#obj function, least distance between the furthest counties?
redistprob += pulp.lpSum([distance(district) * x[district] for district in possible_districts])

# specify the maximum number of districts
redistprob += (
    pulp.lpSum([x[district] for district in possible_districts]) <= max_districts,
    "Maximum_number_of_districts",
)


#Counties in the same congressional district must be adjacent
#List of counties in district, check adjacent counties column

#Sum of population per district is within average +/- 5% range
#2 constraints, x< pop average+5%, x> pop average 05%

#At least one county per district
# A guest must seated at one and only one table
#guest are counties
#tables are districts
for county in counties:
    redistprob += (
        pulp.lpSum([x[district] for district in possible_districts if county in district]) >= 1,
        f"Must_seat_{county}",
    )
    
#Every county must be in exactly one district


#Maximum number of counties per district (potentially)
#Maximum distance between furthest counties in a district (potentially)
#Proportional number of majority Democrat/Republican districts to popular split of Democratic and Republican votes from 2020/2022




In [23]:
#####
import pulp

max_tables = 5
max_table_size = 4
guests = "A B C D E F G I J K L M N O P Q R".split()


def happiness(table):
    """
    Find the happiness of the table
    - by calculating the maximum distance between the letters
    """
    return abs(ord(table[0]) - ord(table[-1]))


# create list of all possible tables
possible_tables = [tuple(c) for c in pulp.allcombinations(guests, max_table_size)]

# create a binary variable to state that a table setting is used
x = pulp.LpVariable.dicts(
    "table", possible_tables, lowBound=0, upBound=1, cat=pulp.LpInteger
)

seating_model = pulp.LpProblem("Wedding Seating Model", pulp.LpMinimize)

seating_model += pulp.lpSum([happiness(table) * x[table] for table in possible_tables])

# specify the maximum number of tables
seating_model += (
    pulp.lpSum([x[table] for table in possible_tables]) <= max_tables,
    "Maximum_number_of_tables",
)

# A guest must seated at one and only one table
for guest in guests:
    seating_model += (
        pulp.lpSum([x[table] for table in possible_tables if guest in table]) == 1,
        f"Must_seat_{guest}",
    )

seating_model.solve()

print(f"The chosen tables are out of a total of {len(possible_tables)}:")
for table in possible_tables:
    if x[table].value() == 1.0:
        print(table)
        

/opt/anaconda3/lib/python3.12/site-packages/pulp/pulp.py:1298: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /opt/anaconda3/lib/python3.12/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/p4/prvgygc14vd8rmhsv2qx4rxc0000gn/T/2fa5c27e52354c9cb29a396b81a52e9c-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/p4/prvgygc14vd8rmhsv2qx4rxc0000gn/T/2fa5c27e52354c9cb29a396b81a52e9c-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 23 COLUMNS
At line 24708 RHS
At line 24727 BOUNDS
At line 27941 ENDATA
Problem MODEL has 18 rows, 3213 columns and 15062 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 12 - 0.01 seconds
Cgl0004I processed model has 18 rows, 3213 columns (3213 integer (3213 of which binary)) and 15062 elements
Cutoff increment increased from 1e-05 to 0.9999
Cbc0038I Initial state - 0 integers unsatisfied sum - 0
Cbc0038I Solution found of 12
Cbc0038I Before mini

## Grace's Attempt

In [6]:
## attempt 3

#num_districts
districts = 17

# max number of counties per district - probably should be increased but its really heavy to run
max_district_size = 5

# objective function?
def happiness(table):
    """
    Find the happiness of the table
    - by calculating the maximum distance between the letters
    """
    return abs(ord(table[0]) - ord(table[-1]))


# create list of all possible districts
possible_districts = [tuple(c) for c in pulp.allcombinations(counties, max_district_size)]

# create a binary variable to state whether or not that district is being used
x = pulp.LpVariable.dicts(
    "district", possible_districts, lowBound=0, upBound=1, cat=pulp.LpInteger
)

redistricting_model = pulp.LpProblem("Wedding Seating Model", pulp.LpMinimize)

# objective function
redistricting_model += pulp.lpSum([happiness(county) * x[county] for county in possible_districts])

# -------------------------------- CONSTRAINTS ---------------------------

# specify the number of districts
redistricting_model += (
    pulp.lpSum([x[county] for county in possible_districts]) == 17,
    "17 districts",
)

# A county must be in exactly one district
for county in counties:
    redistricting_model += (
        pulp.lpSum([x[district] for district in possible_districts if county in district]) == 1,
        f"Must_allocate_{county}",
    )

# TODO: Counties must be adjacent to one another

# TODO: Some type of maximum distance?

redistricting_model.solve()

print(f"The chosen districts are out of a total of {len(possible_districts)}:")
for district in possible_districts:
    if x[county].value() == 1.0:
        print(county)


Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x10680e940>>
Traceback (most recent call last):
  File "/Users/gracefujinaga/Library/Python/3.9/lib/python/site-packages/ipykernel/ipkernel.py", line 770, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrupt: 


The above code took 27+ minutes to run just through adding the objective function with very minimal constraints, we might need to pair it down to fewer options and run the code multiple times or use GPU. Just things to note. The code below adds more constraints

In [5]:
## attempt 4

#num_districts
districts = 17

# max number of counties per district - TODO probably should be increased but its really heavy to run
max_district_size = 5

# create list of all possible districts
possible_districts = [tuple(c) for c in pulp.allcombinations(counties, max_district_size)]

# create a binary variable to state whether or not that district is being used
x = pulp.LpVariable.dicts(
    "district", possible_districts, lowBound=0, upBound=1, cat=pulp.LpInteger
)

redistricting_model = pulp.LpProblem("Redistricting Model", pulp.LpMinimize)

# Objective: Minimize the squared difference between district populations and the target population
#           - squared because we want all positive values when summing it 

# total_pop calculated above
target_population = total_pop/17

redistricting_model += pulp.lpSum(
    (pulp.lpSum(county_population[county] for county in district) - target_population) ** 2 * x[district]
    for district in possible_districts
), "Minimize_Squared_Difference"

# -------------------------------- CONSTRAINTS ---------------------------

# specify the number of districts
redistricting_model += (
    pulp.lpSum([x[county] for county in possible_districts]) == 17,
    "17 districts",
)

# A county must be in exactly one district
for county in counties:
    redistricting_model += (
        pulp.lpSum([x[district] for district in possible_districts if county in district]) == 1,
        f"Must_allocate_{county}",
    )

# Counties must be adjacent to one another
for district in possible_districts:
    for i, county in enumerate(district):
        neighbors = set(adj_list[county])

        # Check if all counties in the district have at least one neighbor in the district
        if not any(neighbor in district for neighbor in neighbors if neighbor != county):
            redistricting_model += x[district] == 0, f"Contiguity_failed:_{district}_{county}"

# TODO: Some type of maximum distance?

redistricting_model.solve()

print(f"The chosen districts are out of a total of {len(possible_districts)}:")
for district in possible_districts:
    if x[county].value() == 1.0:
        print(county)
